In [1]:
import numpy as np
import json, nltk
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt
from json import JSONEncoder
import numpy
import os

In [3]:
split = "val"

In [4]:
wdir = '/media/arnau/PEN/TFG/data_assesing/'
source = f'/media/arnau/PEN/TFG/{split}/'

In [5]:
def load_data(wdir):
    
    # load vqa annotations
    vqa_annot = {'train': {}, 'val': {}, 'test': {}}
    for split in ['train', 'val', 'test']:
        with open(wdir + 'annotations/vqa_annotations/{}.json'.format(split), encoding="utf8") as file:
            annot = json.load(file)
            for entry in annot:
                temp = {'question': entry['question']} 
                if split != 'test':
                    temp['answerable'] = float(entry['answerable'])
                vqa_annot[split][entry['image']] = temp
                
    THRESHOLD = 2
    # load quality annotations
    quality_annot = {'train': {}, 'val': {}, 'test': {}}
    for split in ['train', 'val', 'test']:
        with open(wdir + 'annotations/quality_annotations/{}.json'.format(split)) as file:
            annot = json.load(file)
            for entry in annot:
                temp = {} 
                if split != 'test':
                    flaws = entry['flaws']
                    #print(flaws.keys())
                    temp['flaws'] = np.array(list(flaws.values())) >= THRESHOLD
                    temp['recognizable'] = float(1 - (entry['unrecognizable'] >= THRESHOLD))
                quality_annot[split][entry['image']] = temp
                
    merged_annot = {'train': {}, 'val': {}, 'test': {}}
    for split in ['train', 'val']:
        vqa_split, quality_split = vqa_annot[split], quality_annot[split]
        for fname in vqa_split:
            if quality_split.get(fname):
                merged_annot[split][fname] = {**vqa_split[fname], **quality_split[fname]}
                
    return merged_annot

dict_keys(['FRM', 'BLR', 'DRK', 'BRT', 'OBS', 'OTH', 'NON', 'ROT'])

In [6]:
dicc = load_data(wdir)

In [7]:
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)
    
with open(f"../../data/final_{split}.json", "w") as outf:
    data = json.dump(dicc, outf, cls=NumpyArrayEncoder) 